In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
from astropy.io import fits
from astropy.table import Table
import numpy as np
import matplotlib.pyplot as plt
import glob
import shutil
from tqdm import tqdm

from photutils.segmentation import SegmentationImage

from cutout import cutout_worker

## For isolated

In [11]:
img_name = 'nircam6'
img_base = f'./io/sample/{img_name}/'
img_dirs = glob.glob(f'{img_base}/*/')
img_dirs.sort()

prep_path = f'./io/prep/{img_name}/pregalfit.fits'
with fits.open(prep_path) as hdul:
    tab_img = Table(hdul['IMGLIST'].data)
    tab_ini = Table(hdul['INIPARAM'].data)
    segm = SegmentationImage(hdul['SEGMAP'].data)
areas = segm.areas

patch_base = f'./patch/{img_name}/'

In [12]:
#raise ValueError('This will overwrite existing files. Comment out this line to proceed.')
for img_dir in tqdm(img_dirs):
    id = img_dir.split('/')[-2]
    row = tab_ini[tab_ini['ID'] == int(id)][0]
    size = fits.getdata(f'{img_dir}/sci_F115W.fits').shape[0]
    cutout_worker(id, row['xcentroid'], row['ycentroid'], int(row['label']), size, tab_img, segm, areas, patch_base, overwrite=True)

100%|██████████| 74/74 [06:35<00:00,  5.34s/it]


### For those abnormal cases

In [82]:
id = '18384'
img_dir = f'./io/sample/{img_name}/{id}/'
row = tab_ini[tab_ini['ID'] == int(id)][0]
size = fits.getdata(f'{img_dir}/sci_F115W.fits').shape[0]
print(size)
size = 138
cutout_worker(id, row['xcentroid'], row['ycentroid'], [int(row['label'])], size, tab_img, segm, areas, patch_base, overwrite=True)

220


## Make patches to multicomp? How?

In [19]:
img_name = 'nircam6'
img_base = f'./io/sample/{img_name}/'
img_dirs = glob.glob(f'{img_base}/*/')
img_dirs.sort()

prep_path = f'./io/prep/{img_name}/pregalfit.fits'
with fits.open(prep_path) as hdul:
    tab_img = Table(hdul['IMGLIST'].data)
    tab_ini = Table(hdul['INIPARAM'].data)
    segm = SegmentationImage(hdul['SEGMAP'].data)
areas = segm.areas

patch_base = './patch/multicomp/'

In [20]:
mcomp_base = './multicomp/'
mcomp_dirs = glob.glob(mcomp_base + '*/')
mcomp_dirs.sort()
multilog_path = f'multicomp_{img_name}.fits'
tab_multi = Table(names=['ID','NIRCamPointing','positions','size'], dtype=[str, str, list, float])
for mcomp_dir in mcomp_dirs[:-1]:
    id = mcomp_dir.split('/')[-2]
    idx = np.where(tab_ini['ID'] == int(id))[0]
    if len(idx) == 0:
        continue
    else:
        row = tab_ini[idx][0]
        fdme_path = f'{mcomp_dir}{id}_F115W.galfit'
        with fits.open(f'{mcomp_dir}sci_F115W.fits') as hdul:
            hdr = hdul[0].header
            size = hdul[0].data.shape[0]
        xc, yc = int(hdr['xc']), int(hdr['yc'])
        
        with open(fdme_path, 'r') as f:
            cnt = -1
            positions = []
            for line in f:
                if line.startswith(' 0) sersic'):
                    cnt = 3
                if cnt == 2:
                    posx = int(line.split()[1]) - 1
                    pass
                if cnt == 1:
                    posy = int(line.split()[1]) - 1
                    pass
                if cnt == 0:
                    positions.append((posx,posy))
                cnt = cnt-1
            
            labels = list({segm.data[pos[1]+yc-size//2, pos[0]+xc-size//2] for pos in positions})
        
        status = cutout_worker(id, xc, yc, labels, size, tab_img, segm, areas, patch_base, overwrite=True)
        if status == 1:
            shutil.copy2(f'{mcomp_dir}{id}_multicomp.galfit', f'{patch_base}{id}/{id}_multicomp.galfit')
        else: 
            print('please manually adjust: ', id)
        tab_multi.add_row([id, img_name, positions, size])
tab_multi.write('./patch/multicomp/' + multilog_path)